In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from PIL import Image

In [23]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # convolutional layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # fully connected layers
        # 64 * 7 * 7 is the output size of the last convolutional layer
        # output of conv1 is size 32 * 28 * 28
        # after pool1, the size is 32 * 14 * 14
        # output of conv2 is size 64 * 14 * 14
        # after pool2, the size is 64 * 7 * 7
        # 128 is the number of neurons in the first fully connected layer
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)  # flatten
        x = self.relu3(self.fc1(x))  # fully connected layer -> ReLU
        x = self.fc2(x)  # final layer
        return x

In [24]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),  # converts PIL Image or numpy.ndarray to tensor
        transforms.Normalize((0.5,), (0.5,)),  # normalize pixel values to [-1, 1]
    ]
)

train_dataset = torchvision.datasets.MNIST(
    root="../data", train=True, download=True, transform=transform
)
test_dataset = torchvision.datasets.MNIST(
    root="../data", train=False, download=True, transform=transform
)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

In [25]:
def train_model(model, criterion, optimizer, device, epochs=5):
    print("Training model...")
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader):.4f}')

In [26]:
def test_model(model, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Test Accuracy: {100 * correct / total:.2f}%')

In [27]:
def save_model(model, path):
    torch.save(model.state_dict(), path)
    print(f'Model saved to {path}')

In [28]:
def load_model(path, device):
    model = CNN()
    model.load_state_dict(torch.load(path, map_location=device, weights_only=True))
    model.to(device)
    model.eval()
    print(f'Model loaded from {path}')
    return model

In [29]:
def predict_image(model, image_path, device):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    image = Image.open(image_path).convert('L')  # Convert to grayscale
    image = image.resize((28, 28))  # Resize to 28x28
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension

    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)

    print(f'Predicted Label: {predicted.item()}')
    return predicted.item()

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [31]:
def train_and_test():
        
    # init model, loss function, and optimizer
    model = CNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # train model
    train_model(model, criterion, optimizer, device, epochs=5)
    
    # test model
    test_model(model, device)
    
    # save model
    save_model(model, '../models/mnist_cnn.pth')

In [32]:
def predict_image_from_file(model_path, image_path):
    model = load_model(model_path, device)
    predict_image(model, image_path, device)

In [33]:
train_and_test()

Training model...


KeyboardInterrupt: 

In [34]:
predict_image_from_file('../trained_models/emnist_cnn_balanced_86.93.pth', '../data/image.png')

RuntimeError: Error(s) in loading state_dict for CNN:
	size mismatch for fc2.weight: copying a param with shape torch.Size([62, 128]) from checkpoint, the shape in current model is torch.Size([10, 128]).
	size mismatch for fc2.bias: copying a param with shape torch.Size([62]) from checkpoint, the shape in current model is torch.Size([10]).